In [1]:
# # # Error: Path resolution
# %%bash

# outd="$(pwd)"
# cgud="../structures/charmm-gui/jobid-0147660438"

# cd "${cgud}"
# echo "$(pwd)"
# python openmm/openmm_run.py \
#     -i "$(pwd)/step5_production.inp" \
#     -p "$(pwd)/step3_pbcsetup.psf" \
#     -c "$(pwd)/step3_pbcsetup.crd" \
#     -b "$(pwd)/step3_pbcsetup.str" \
#     -t "$(pwd)/toppar.str" \
#     -odcd "${outd}/3vte_apo.cgui.dcd" \
#     -opdb "${outd}/3vte_apo.cgui.pdb" \
#     -hmr 

In [8]:
%%bash

# # Parameterize CBGA COO(-)
antechamber \
    -fi pdb -i ../structures/CBGA.COO.DOCK.1.61.pdb \
    -fo mol2 -o ./CBGA.COO.1.61.gaff.mol2 \
    -rn LIG -c bcc -at gaff2 -j 4 -pf y \
    -nc -1
parmchk2 \
    -i ./CBGA.COO.1.61.gaff.mol2 \
    -f mol2 -o CBGA.COO.1.61.frcmod -s 2

# # Make local copies
obabel \
    -imol2 ../structures/CBGA.COO.DOCK.1.61.mol2 \
    -osdf > LIG.sdf
obabel \
    -imol2 ../structures/CBGA.COO.DOCK.1.61.mol2 \
    -opdb > LIG.pdb


Welcome to antechamber 20.0: molecular input file processor.

acdoctor mode is on: check and diagnose problems in the input file.
-- Check Format for pdb File --
   Status: pass
-- Check Unusual Elements --
   Status: pass
-- Check Open Valences --
   Status: pass
-- Check Geometry --
      for those bonded   
      for those not bonded   
   Status: pass
-- Check Weird Bonds --
   Status: pass
-- Check Number of Units --
   Status: pass
acdoctor mode has completed checking the input file.

Info: Total number of electrons: 196; net charge: -1

Running: /Users/sethmartinez/opt/anaconda3/envs/qcmm/bin/sqm -O -i sqm.in -o sqm.out

1 molecule converted
1 molecule converted


In [3]:
import parmed as pmd

from simtk import openmm, unit
from simtk.openmm import app, LangevinIntegrator
from simtk.openmm.app import PDBFile, NoCutoff, HBonds

from openforcefield.topology import Molecule
from openmmforcefields.generators import SystemGenerator
from openforcefield.typing.engines.smirnoff import ForceField

# # Load Parsley force field
offLIG_ff = ForceField('openff-1.2.1.offxml')

ommCBGA = PDBFile('LIG.pdb')
molCBGA = Molecule('LIG.sdf')
ligand_system = offLIG_ff.create_openmm_system(
    molCBGA.to_topology())
pmdCBGA = pmd.openmm.load_topology(
    ommCBGA.topology,
    ligand_system,
    xyz=ommCBGA.positions)

In [5]:
# from parmed import Structure, topologyobjects, load_file
# from parmed.exceptions import ParameterError
# crd3 = load_file(f'{parmslug}.rst7')
# parm3 = readparm.LoadParm(f'{parmslug}.parm7', xyz=crd3.coordinates)

# # Load the Amber restart from CHARMM-GUI's glycan modeller
prmslug = '../structures/charmm-gui/jobid-0147660438/amber/step3_input'
prm3VTE = pmd.amber.AmberParm(
    f'{prmslug}.parm7', f'{prmslug}.rst7')
apo_system = prm3VTE.createSystem()

# # Apo-enzyme in octahedral water, d=10
pmd3VTE = pmd.openmm.load_topology(
    prm3VTE.topology,
    apo_system,
    xyz=prm3VTE.positions)

/Users/sethmartinez/opt/anaconda3/envs/qcmm/lib/python3.6/site-packages/parmed/openmm/topsystem.py:187: OpenMMWarning: Unsupported Force type CustomNonbondedForce
  warnings.warn(&#39;Unsupported Force type %s&#39; % type(force).__name__, OpenMMWarning)


In [7]:
# # Holo-enzyme
pmdCPLX = pmd3VTE + pmdCBGA
pmdCPLX.visualize()

NGLWidget()

In [ ]:
# # # Dummy simulation
# integrator = LangevinIntegrator(
#     302.5*unit.kelvin, 
#     1/unit.picosecond, 
#     0.002*unit.picoseconds)

# simulation = app.Simulation(
#   pmdCPLX.topology, pmdCPLX.getPositions(), integrator)
# simulation.context.setPositions(pmdCPLX.positions)
# simulation.minimizeEnergy()